In [38]:
import os
import torch
import xml.etree.ElementTree as ET
from collections import defaultdict
import warnings

# 커널 충돌 방지용
warnings.filterwarnings("ignore", category=FutureWarning)
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

model_path = r'..\yolov5\runs\train\pcb_final_run\weights\best.pt'
test_list_txt = r'..\VOC_PCB\test.txt'
anno_dir = r'..\VOC_PCB\Annotations'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [39]:
# IoU 계산 함수
def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0

In [40]:
# 테스트 데이터 리스트 확보
if os.path.exists(test_list_txt):
    with open(test_list_txt, 'r') as f:
        test_img_paths = [line.strip() for line in f.readlines() if line.strip()]
    print(f"테스트 이미지 {len(test_img_paths)}개를 로드했습니다.")
else:
    print(f"오류: {test_list_txt} 파일을 찾을 수 없습니다.")
    test_img_paths = []

테스트 이미지 2134개를 로드했습니다.


In [41]:
# 모델 로드
try:
    model = torch.hub.load(r'..\yolov5', 'custom', path=model_path, source='local')
    model.to(device)
    model.conf = 0.25
    print("모델 로드 완료. 정밀 분석을 시작합니다...")
except Exception as e:
    print(f"모델 로드 중 오류 발생: {e}")
    exit()

YOLOv5  2026-1-23 Python-3.12.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


모델 로드 완료. 정밀 분석을 시작합니다...


In [42]:
# 확장된 평가 지표 변수
stats = {
    'total_images': 0,
    'total_gt_objects': 0,     # 실제 결함 총합
    'true_positives': 0,      # 정확히 찾은 결함
    'false_positives': 0,     # 결함이 아닌데 찾았다고 한 경우
    'loc_failures': 0,        # 클래스는 맞으나 위치가 부정확 (IoU < 0.5)
    'class_mismatches': defaultdict(int), # 클래스를 잘못 예측한 경우
    'missed_detections': defaultdict(int), # 아예 못 찾고 지나간 결함
    'iou_scores': []          # 성공한 탐지의 IoU 값들
}

In [43]:
# 분석 루프
for img_path in test_img_paths:
    if not os.path.exists(img_path): continue
    
    stats['total_images'] += 1
    pure_name = os.path.splitext(os.path.basename(img_path))[0]
    xml_path = os.path.join(anno_dir, f"{pure_name}.xml")
    
    if not os.path.exists(xml_path): continue

    # XML에서 실제 정답(GT) 정보 추출
    gt_list = []
    tree = ET.parse(xml_path)
    for obj in tree.findall('object'):
        cls_name = obj.find('name').text
        bndbox = obj.find('bndbox')
        box = [
            int(float(bndbox.find('xmin').text)),
            int(float(bndbox.find('ymin').text)),
            int(float(bndbox.find('xmax').text)),
            int(float(bndbox.find('ymax').text))
        ]
        gt_list.append({'name': cls_name, 'box': box, 'matched': False})
        stats['total_gt_objects'] += 1

    # 모델 추론
    results = model(img_path)
    pred_df = results.pandas().xyxy[0]
    
    # 예측 결과 분석
    for _, pred in pred_df.iterrows():
        pred_box = [pred['xmin'], pred['ymin'], pred['xmax'], pred['ymax']]
        pred_cls = pred['name']
        
        best_iou = 0
        best_gt_idx = -1
        
        # 현재 예측값과 가장 잘 맞는 GT 찾기
        for idx, gt in enumerate(gt_list):
            if gt['matched']: continue
            iou = calculate_iou(pred_box, gt['box'])
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = idx
        
        # 매칭 결과 분류
        if best_gt_idx != -1 and best_iou > 0.1: # 최소 겹침 기준
            gt_obj = gt_list[best_gt_idx]
            if pred_cls == gt_obj['name']:
                if best_iou >= 0.5: # 성공 기준 (IoU 0.5)
                    stats['true_positives'] += 1
                    stats['iou_scores'].append(best_iou)
                    gt_list[best_gt_idx]['matched'] = True
                else:
                    stats['loc_failures'] += 1
            else:
                stats['class_mismatches'][gt_obj['name']] += 1
        else:
            stats['false_positives'] += 1 # 배경을 결함으로 오인

    # Missed 집계
    for gt in gt_list:
        if not gt['matched']:
            stats['missed_detections'][gt['name']] += 1

In [44]:
# 최종 리포트 출력
total_gt = stats['total_gt_objects']
tp = stats['true_positives']
avg_iou = sum(stats['iou_scores']) / len(stats['iou_scores']) if stats['iou_scores'] else 0

print("="*30)
print(f"PCB 결함 탐지 정밀 성능 평가 리포트")
print("="*30)
print(f"1. 전체 테스트 이미지: {stats['total_images']}장")
print(f"2. 실제 결함 총수(GT): {total_gt}개")
print(f"3. 탐지 성공(TP): {tp}개, 재현율: {(tp/total_gt*100 if total_gt > 0 else 0):.2f}%")
print(f"4. 평균 Io수(정확도): {avg_iou:.4f}") # 성공 탐지 기준
print("-" * 30)
print(f"실패 유형 분석")
print(f" - 미탐지(Missed): {sum(stats['missed_detections'].values())}건") # 결함을 못 보고 지나침
print(f" - 오탐지(False Positive): {stats['false_positives']}건") # 결함이 없는데 있다고 함
print(f" - 위치오류(Low IoU): {stats['loc_failures']}건") # 찾았으나 박스 위치가 부정확함
print("-" * 30)
print(f"클래스별 상세 미탐지 현황")
for cls, count in stats['missed_detections'].items():
    print(f" - {cls}: {count}회 탐지 실패")
print("="*30)

PCB 결함 탐지 정밀 성능 평가 리포트
1. 전체 테스트 이미지: 2134장
2. 실제 결함 총수(GT): 4349개
3. 탐지 성공(TP): 4317개, 재현율: 99.26%
4. 평균 Io수(정확도): 0.8217
------------------------------
실패 유형 분석
 - 미탐지(Missed): 32건
 - 오탐지(False Positive): 101건
 - 위치오류(Low IoU): 3건
------------------------------
클래스별 상세 미탐지 현황
 - short: 8회 탐지 실패
 - spur: 14회 탐지 실패
 - spurious_copper: 3회 탐지 실패
 - mouse_bite: 2회 탐지 실패
 - open_circuit: 4회 탐지 실패
 - missing_hole: 1회 탐지 실패


In [45]:
# [추가 코드] 결과 저장 로직
save_path = r"..\qa\model_qa_results"
if not os.path.exists(save_path):
    os.makedirs(save_path)

file_name = os.path.join(save_path, "fnfp_test_report.txt")

with open(file_name, 'w', encoding='utf-8') as f:
    f.write("=" * 30 + "\n")
    f.write("PCB 결함 탐지 정밀 성능 평가 리포트\n")
    f.write("=" * 30 + "\n")
    f.write(f"1. 전체 테스트 이미지: {stats['total_images']}장\n")
    f.write(f"2. 실제 결함 총수(GT): {total_gt}개\n")
    f.write(f"3. 탐지 성공(TP): {tp}개, 재현율: {(tp/total_gt*100 if total_gt > 0 else 0):.2f}%\n")
    f.write(f"4. 평균 IoU(정확도): {avg_iou:.4f}\n")
    f.write("-" * 30 + "\n")
    f.write("실패 유형 분석\n")
    f.write(f" - 미탐지(Missed): {sum(stats['missed_detections'].values())}건\n")
    f.write(f" - 오탐지(False Positive): {stats['false_positives']}건\n")
    f.write(f" - 위치오류(Low IoU): {stats['loc_failures']}건\n")
    f.write("-" * 30 + "\n")
    f.write("클래스별 상세 미탐지 현황\n")
    for cls, count in stats['missed_detections'].items():
        f.write(f" - {cls}: {count}회 탐지 실패\n")
    f.write("=" * 30 + "\n")

print(f"정밀 성능 평가 결과가 저장되었습니다: {file_name}")

정밀 성능 평가 결과가 저장되었습니다: ..\qa\model_qa_results\fnfp_test_report.txt
